In [1]:
import pandas as pd
import requests
from tqdm.auto import tqdm
tqdm.pandas()

In [2]:
def next_monday(date):
    return pd.date_range(start=date, end=date + pd.offsets.Day(6), freq='W-MON')[0]

# Setting

In [3]:
test_date = pd.to_datetime('2021-05-24')

In [4]:
models_to_exclude = ['COVIDhub-ensemble', 'COVIDhub-trained_ensemble', 
                     'CU-nochange', 'CU-scenario_high', 'CU-scenario_low', 'CU-scenario_mid']

locations_to_exclude = ["60", "66", "69", "72", "74", "78"]

# DC,11,District of Columbia
# AS,60,American Samoa
# GU,66,Guam
# MP,69,Northern Mariana Islands
# PR,72,Puerto Rico
# UM,74,U.S. Minor Outlying Islands
# VI,78,Virgin Islands

In [5]:
dtype={'target': str, 'location': str, 'type': str, 'quantile': float, 'value': float}
parse_dates=['forecast_date', 'target_end_date']

# Load Files

In [6]:
url = "https://api.github.com/repos/reichlab/covid19-forecast-hub/git/trees/master?recursive=1"
r = requests.get(url)
res = r.json()

In [7]:
files = [file["path"] for file in res["tree"] if (file["path"].startswith('data-processed/') and file["path"].endswith('.csv'))]

In [8]:
files[-5:]

['data-processed/epiforecasts-ensemble1/2021-04-26-epiforecasts-ensemble1.csv',
 'data-processed/epiforecasts-ensemble1/2021-05-03-epiforecasts-ensemble1.csv',
 'data-processed/epiforecasts-ensemble1/2021-05-10-epiforecasts-ensemble1.csv',
 'data-processed/epiforecasts-ensemble1/2021-05-17-epiforecasts-ensemble1.csv',
 'data-processed/epiforecasts-ensemble1/2021-05-24-epiforecasts-ensemble1.csv']

In [9]:
df_files = pd.DataFrame({'filename':files})

df_files['model'] = df_files.filename.apply(lambda f: f.split('/')[1])

df_files['forecast_date'] = df_files.filename.apply(lambda f: f.split('/')[2][:10])
df_files.forecast_date = pd.to_datetime(df_files.forecast_date)

df_files['timezero'] = df_files.forecast_date.apply(next_monday)

df_files = df_files[~df_files.model.isin(models_to_exclude)]

In [10]:
df_files

,filename,model,forecast_date,timezero
0,data-processed/AIpert-pwllnod/2020-12-21-AIper...,AIpert-pwllnod,2020-12-21,2020-12-21
1,data-processed/AIpert-pwllnod/2020-12-28-AIper...,AIpert-pwllnod,2020-12-28,2020-12-28
2,data-processed/AIpert-pwllnod/2021-01-04-AIper...,AIpert-pwllnod,2021-01-04,2021-01-04
3,data-processed/AIpert-pwllnod/2021-01-11-AIper...,AIpert-pwllnod,2021-01-11,2021-01-11
4,data-processed/AIpert-pwllnod/2021-01-18-AIper...,AIpert-pwllnod,2021-01-18,2021-01-18
...,...,...,...,...
3480,data-processed/epiforecasts-ensemble1/2021-04-...,epiforecasts-ensemble1,2021-04-26,2021-04-26
3481,data-processed/epiforecasts-ensemble1/2021-05-...,epiforecasts-ensemble1,2021-05-03,2021-05-03
3482,data-processed/epiforecasts-ensemble1/2021-05-...,epiforecasts-ensemble1,2021-05-10,2021-05-10
3483,data-processed/epiforecasts-ensemble1/2021-05-...,epiforecasts-ensemble1,2021-05-17,2021-05-17


### Test Data

In [11]:
VALID_TARGETS = [f"{_} wk ahead inc death" for _ in range(1, 5)] + \
                [f"{_} wk ahead cum death" for _ in range(1, 5)] + \
                [f"{_} wk ahead inc case" for _ in range(1, 5)]

In [12]:
df_test_files = df_files[df_files.timezero == test_date]

In [13]:
df_test_files.head()

,filename,model,forecast_date,timezero
22,data-processed/AIpert-pwllnod/2021-05-24-AIper...,AIpert-pwllnod,2021-05-24,2021-05-24
68,data-processed/BPagano-RtDriven/2021-05-23-BPa...,BPagano-RtDriven,2021-05-23,2021-05-24
117,data-processed/CEID-Walk/2021-05-24-CEID-Walk.csv,CEID-Walk,2021-05-24,2021-05-24
162,data-processed/CMU-TimeSeries/2021-05-24-CMU-T...,CMU-TimeSeries,2021-05-24,2021-05-24
222,data-processed/COVIDhub-baseline/2021-05-24-CO...,COVIDhub-baseline,2021-05-24,2021-05-24


In [14]:
dfs_test = []

for _, row in tqdm(df_test_files.iterrows(), total=df_test_files.shape[0]):
    #print(row['filename'])
    df_temp = pd.read_csv('https://github.com/reichlab/covid19-forecast-hub/raw/master/' + row['filename'],
                         dtype=dtype, parse_dates=parse_dates)
    df_temp = df_temp[df_temp.target.isin(VALID_TARGETS)]
    df_temp['model'] = row['model']
    dfs_test.append(df_temp)

  0%|          | 0/48 [00:00<?, ?it/s]

In [15]:
df_test = pd.concat(dfs_test)

In [16]:
df_test.shape

(1851723, 8)

In [17]:
df_test

,forecast_date,target,target_end_date,quantile,type,value,location,model
0,2021-05-24,1 wk ahead cum death,2021-05-29,0.025,quantile,591099.187840,US,AIpert-pwllnod
1,2021-05-24,1 wk ahead cum death,2021-05-29,0.250,quantile,592611.396957,US,AIpert-pwllnod
2,2021-05-24,1 wk ahead cum death,2021-05-29,0.750,quantile,594510.605904,US,AIpert-pwllnod
3,2021-05-24,1 wk ahead cum death,2021-05-29,0.975,quantile,596617.985106,US,AIpert-pwllnod
4,2021-05-24,1 wk ahead cum death,2021-05-29,NaN,point,593404.857658,US,AIpert-pwllnod
...,...,...,...,...,...,...,...,...
10939,2021-05-24,4 wk ahead cum death,2021-06-19,0.850,quantile,599358.000000,US,epiforecasts-ensemble1
10940,2021-05-24,4 wk ahead cum death,2021-06-19,0.900,quantile,599598.000000,US,epiforecasts-ensemble1
10941,2021-05-24,4 wk ahead cum death,2021-06-19,0.950,quantile,599909.000000,US,epiforecasts-ensemble1
10942,2021-05-24,4 wk ahead cum death,2021-06-19,0.975,quantile,600194.000000,US,epiforecasts-ensemble1


In [18]:
# only consider US + 50 states
df_test = df_test[df_test.location.str.len() == 2]
df_test = df_test[~df_test.location.isin(locations_to_exclude)]

In [19]:
df_test.shape

(345054, 8)

In [20]:
df_test.location.nunique()

52

In [21]:
# ensure that for all targets each model provides forecasts for all locations
df_test = df_test[df_test.groupby(['target', 'model'])['location'].transform('nunique') == 52]

In [22]:
df_test = df_test[df_test.type == 'quantile']

In [23]:
df_test['no_quantiles'] = df_test.groupby(['model', 'target', 'target_end_date', 'location'])['quantile'].transform('nunique')

df_test['no_quantiles'] = df_test.groupby(['target', 'model'])['no_quantiles'].transform('min')

In [24]:
df_test = df_test[(df_test.no_quantiles == 23) | 
        (df_test.target.str.contains('inc case') & (df_test.no_quantiles == 7))].drop(columns='no_quantiles').reset_index(drop=True)

In [25]:
df_test.shape

(246272, 8)

In [26]:
# dict of the models available for each target
available_models = dict(df_test.groupby(['target'])['model'].unique())

In [27]:
available_models

{'1 wk ahead cum death': array(['BPagano-RtDriven', 'CEID-Walk', 'COVIDhub-baseline',
        'Covid19Sim-Simulator', 'CovidAnalytics-DELPHI', 'DDS-NBDS',
        'IHME-CurveFit', 'JHUAPL-Bucky', 'JHU_CSSE-DECOM',
        'JHU_IDD-CovidSP', 'Karlen-pypm', 'LANL-GrowthRate', 'LNQ-ens1',
        'MIT_CritData-GBCF', 'Microsoft-DeepSTIA', 'RobertWalraven-ESG',
        'SteveMcConnell-CovidComplete', 'UA-EpiCovDA', 'UMass-MechBayes',
        'USC-SI_kJalpha', 'epiforecasts-ensemble1'], dtype=object),
 '1 wk ahead inc case': array(['BPagano-RtDriven', 'CEID-Walk', 'COVIDhub-baseline',
        'Covid19Sim-Simulator', 'CovidAnalytics-DELPHI',
        'IEM_MED-CovidProject', 'JHUAPL-Bucky', 'JHU_CSSE-DECOM',
        'JHU_IDD-CovidSP', 'Karlen-pypm', 'LANL-GrowthRate', 'LNQ-ens1',
        'MUNI-ARIMA', 'Microsoft-DeepSTIA', 'RobertWalraven-ESG',
        'USC-SI_kJalpha', 'UVA-Ensemble'], dtype=object),
 '1 wk ahead inc death': array(['BPagano-RtDriven', 'CEID-Walk', 'COVIDhub-baseline',
       

## Training Data

d - 4 weeks - (horizon - 1) --> end: d - horizon

In [28]:
lower_bound = test_date - pd.Timedelta(weeks=4) - pd.Timedelta(weeks=(4 - 1))

In [29]:
lower_bound

Timestamp('2021-04-05 00:00:00')

In [30]:
df_files = df_files[(df_files.timezero >= lower_bound) & (df_files.timezero < test_date)].copy()

In [31]:
# assigns to each horizon the corresponding training forecast dates for the test date
h_dict = {}
for h in range(1, 5):
    h_dict[h] = [test_date - pd.Timedelta(weeks=4) - pd.Timedelta(weeks=(h - 1)), 
                 test_date - pd.Timedelta(weeks=h)]

In [32]:
h_dict

{1: [Timestamp('2021-04-26 00:00:00'), Timestamp('2021-05-17 00:00:00')],
 2: [Timestamp('2021-04-19 00:00:00'), Timestamp('2021-05-10 00:00:00')],
 3: [Timestamp('2021-04-12 00:00:00'), Timestamp('2021-05-03 00:00:00')],
 4: [Timestamp('2021-04-05 00:00:00'), Timestamp('2021-04-26 00:00:00')]}

In [33]:
def relevant_horizons(d):
    hs = []
    for h in range(1, 5):
        if((d >= h_dict[h][0]) & (d <= h_dict[h][1])):
            hs.append(h)
    return hs

In [34]:
df_files['horizons'] = df_files.timezero.apply(relevant_horizons)

In [35]:
df_files

,filename,model,forecast_date,timezero,horizons
15,data-processed/AIpert-pwllnod/2021-04-05-AIper...,AIpert-pwllnod,2021-04-05,2021-04-05,[4]
16,data-processed/AIpert-pwllnod/2021-04-12-AIper...,AIpert-pwllnod,2021-04-12,2021-04-12,"[3, 4]"
17,data-processed/AIpert-pwllnod/2021-04-19-AIper...,AIpert-pwllnod,2021-04-19,2021-04-19,"[2, 3, 4]"
18,data-processed/AIpert-pwllnod/2021-04-26-AIper...,AIpert-pwllnod,2021-04-26,2021-04-26,"[1, 2, 3, 4]"
19,data-processed/AIpert-pwllnod/2021-05-03-AIper...,AIpert-pwllnod,2021-05-03,2021-05-03,"[1, 2, 3]"
...,...,...,...,...,...
3479,data-processed/epiforecasts-ensemble1/2021-04-...,epiforecasts-ensemble1,2021-04-19,2021-04-19,"[2, 3, 4]"
3480,data-processed/epiforecasts-ensemble1/2021-04-...,epiforecasts-ensemble1,2021-04-26,2021-04-26,"[1, 2, 3, 4]"
3481,data-processed/epiforecasts-ensemble1/2021-05-...,epiforecasts-ensemble1,2021-05-03,2021-05-03,"[1, 2, 3]"
3482,data-processed/epiforecasts-ensemble1/2021-05-...,epiforecasts-ensemble1,2021-05-10,2021-05-10,"[1, 2]"


In [36]:
# only keep relevant training data
df_files = df_files[df_files.horizons.apply(len) > 0]

In [37]:
dfs = []
for _, row in tqdm(df_files.iterrows(), total=df_files.shape[0]):
    VALID_TARGETS = [f"{_} wk ahead inc death" for _ in row['horizons']] + \
                    [f"{_} wk ahead cum death" for _ in row['horizons']] + \
                    [f"{_} wk ahead inc case" for _ in row['horizons']]
    df_temp = pd.read_csv('https://github.com/reichlab/covid19-forecast-hub/raw/master/' + row['filename'],
                         dtype=dtype, parse_dates=parse_dates)
    df_temp = df_temp[df_temp.target.isin(VALID_TARGETS)]
    df_temp['model'] = row['model']
    dfs.append(df_temp)

  0%|          | 0/418 [00:00<?, ?it/s]

In [38]:
df = pd.concat(dfs)

In [39]:
df.shape

(9448086, 8)

# Data Cleaning

In [40]:
df = df[df.location.str.len() == 2]
df = df[~df.location.isin(locations_to_exclude)]

In [41]:
df.location.nunique() # US + 50 states + district of Columbia

52

In [42]:
df = df[df.type == 'quantile']

In [43]:
# how many forecasts for each target/model/location? should be 4 for every location
df['no_forecasts'] = df.groupby(['target', 'model', 'location'])['target_end_date'].transform('nunique')

In [44]:
df = df[df.groupby(['target', 'model'])['no_forecasts'].transform('min') == 4].drop(columns='no_forecasts').reset_index(drop=True)

In [45]:
# df = df[df.groupby(['target', 'model', 'location'])['target_end_date'].transform('nunique') == 4]
# would only remove the respective location with <4 forecasts, but we want to remove the model as a whole if in one location <4

In [46]:
df = df[df.groupby(['target', 'model', 'target_end_date'])['location'].transform('nunique') == 52]

In [47]:
df['no_quantiles'] = df.groupby(['model', 'target', 'target_end_date', 'location'])['quantile'].transform('nunique')

df['no_quantiles'] = df.groupby(['target', 'model'])['no_quantiles'].transform('min')

In [48]:
df[(df.target.str.contains('inc case') & (df.no_quantiles == 7))]

,forecast_date,target,target_end_date,quantile,type,value,location,model,no_quantiles
9048,2021-04-04,4 wk ahead inc case,2021-05-01,0.025,quantile,385284.712000,US,BPagano-RtDriven,7
9049,2021-04-04,4 wk ahead inc case,2021-05-01,0.100,quantile,477357.042410,US,BPagano-RtDriven,7
9050,2021-04-04,4 wk ahead inc case,2021-05-01,0.250,quantile,561032.461690,US,BPagano-RtDriven,7
9051,2021-04-04,4 wk ahead inc case,2021-05-01,0.500,quantile,654309.447150,US,BPagano-RtDriven,7
9052,2021-04-04,4 wk ahead inc case,2021-05-01,0.750,quantile,747586.432610,US,BPagano-RtDriven,7
...,...,...,...,...,...,...,...,...,...
1059111,2021-05-17,1 wk ahead inc case,2021-05-22,0.250,quantile,116222.876123,US,UVA-Ensemble,7
1059112,2021-05-17,1 wk ahead inc case,2021-05-22,0.500,quantile,193247.603623,US,UVA-Ensemble,7
1059113,2021-05-17,1 wk ahead inc case,2021-05-22,0.750,quantile,270272.331123,US,UVA-Ensemble,7
1059114,2021-05-17,1 wk ahead inc case,2021-05-22,0.900,quantile,339596.988598,US,UVA-Ensemble,7


In [49]:
df = df[(df.no_quantiles == 23) | 
        (df.target.str.contains('inc case') & (df.no_quantiles == 7))].drop(columns='no_quantiles').reset_index(drop=True)

In [50]:
df.groupby('target').model.nunique()

target
1 wk ahead cum death    20
1 wk ahead inc case     17
1 wk ahead inc death    22
2 wk ahead cum death    20
2 wk ahead inc case     16
2 wk ahead inc death    21
3 wk ahead cum death    20
3 wk ahead inc case     15
3 wk ahead inc death    21
4 wk ahead cum death    19
4 wk ahead inc case     16
4 wk ahead inc death    21
Name: model, dtype: int64

In [51]:
# check if there are models used for training that are not available for the test date
train_models = dict(df.groupby('target').model.unique())

a = [(k, t) for k, v in train_models.items() for t in v]
b = [(k, t) for k, v in available_models.items() for t in v]

[i for i in a if i not in b]

[('1 wk ahead cum death', 'CU-select'),
 ('1 wk ahead cum death', 'OliverWyman-Navigator'),
 ('1 wk ahead inc case', 'CU-select'),
 ('1 wk ahead inc case', 'UChicagoCHATTOPADHYAY-UnIT'),
 ('1 wk ahead inc death', 'CU-select'),
 ('1 wk ahead inc death', 'OliverWyman-Navigator'),
 ('2 wk ahead cum death', 'CU-select'),
 ('2 wk ahead cum death', 'OliverWyman-Navigator'),
 ('2 wk ahead inc case', 'CU-select'),
 ('2 wk ahead inc case', 'UChicagoCHATTOPADHYAY-UnIT'),
 ('2 wk ahead inc death', 'CU-select'),
 ('2 wk ahead inc death', 'OliverWyman-Navigator'),
 ('3 wk ahead cum death', 'CU-select'),
 ('3 wk ahead cum death', 'OliverWyman-Navigator'),
 ('3 wk ahead inc case', 'CU-select'),
 ('3 wk ahead inc death', 'CU-select'),
 ('3 wk ahead inc death', 'OliverWyman-Navigator'),
 ('4 wk ahead cum death', 'CU-select'),
 ('4 wk ahead inc case', 'CU-select'),
 ('4 wk ahead inc death', 'CU-select'),
 ('4 wk ahead inc death', 'Columbia_UNC-SurvCon')]

In [52]:
df = df[df.apply(lambda x: x.model in (available_models[x.target]), axis=1)]

In [53]:
df.groupby('target').model.nunique()

target
1 wk ahead cum death    18
1 wk ahead inc case     15
1 wk ahead inc death    20
2 wk ahead cum death    18
2 wk ahead inc case     14
2 wk ahead inc death    19
3 wk ahead cum death    18
3 wk ahead inc case     14
3 wk ahead inc death    19
4 wk ahead cum death    18
4 wk ahead inc case     15
4 wk ahead inc death    19
Name: model, dtype: int64

In [54]:
df.shape

(841360, 8)

In [55]:
models_by_target = pd.DataFrame([(i, k) for i,j in train_models.items() for k in j], 
                  columns=['target','model'])

In [56]:
models_by_target[models_by_target.target == '4 wk ahead cum death']

,target,model
172,4 wk ahead cum death,BPagano-RtDriven
173,4 wk ahead cum death,CEID-Walk
174,4 wk ahead cum death,COVIDhub-baseline
175,4 wk ahead cum death,CU-select
176,4 wk ahead cum death,Covid19Sim-Simulator
177,4 wk ahead cum death,CovidAnalytics-DELPHI
178,4 wk ahead cum death,DDS-NBDS
179,4 wk ahead cum death,JHUAPL-Bucky
180,4 wk ahead cum death,Karlen-pypm
181,4 wk ahead cum death,LANL-GrowthRate


In [57]:
df = df[~df.model.isin(models_to_exclude)]

In [58]:
df

,forecast_date,target,target_end_date,quantile,type,value,location,model
0,2021-04-04,4 wk ahead inc death,2021-05-01,0.010,quantile,3613.39426,US,BPagano-RtDriven
1,2021-04-04,4 wk ahead inc death,2021-05-01,0.025,quantile,4120.81110,US,BPagano-RtDriven
2,2021-04-04,4 wk ahead inc death,2021-05-01,0.050,quantile,4574.41502,US,BPagano-RtDriven
3,2021-04-04,4 wk ahead inc death,2021-05-01,0.100,quantile,5105.57034,US,BPagano-RtDriven
4,2021-04-04,4 wk ahead inc death,2021-05-01,0.150,quantile,5466.27169,US,BPagano-RtDriven
...,...,...,...,...,...,...,...,...
921851,2021-05-17,1 wk ahead cum death,2021-05-22,0.850,quantile,589397.00000,US,epiforecasts-ensemble1
921852,2021-05-17,1 wk ahead cum death,2021-05-22,0.900,quantile,589439.00000,US,epiforecasts-ensemble1
921853,2021-05-17,1 wk ahead cum death,2021-05-22,0.950,quantile,589516.00000,US,epiforecasts-ensemble1
921854,2021-05-17,1 wk ahead cum death,2021-05-22,0.975,quantile,589597.00000,US,epiforecasts-ensemble1


In [59]:
available_train_models = dict(df.groupby(['target'])['model'].unique())

In [60]:
available_train_models

{'1 wk ahead cum death': array(['BPagano-RtDriven', 'CEID-Walk', 'COVIDhub-baseline',
        'Covid19Sim-Simulator', 'CovidAnalytics-DELPHI', 'DDS-NBDS',
        'JHUAPL-Bucky', 'JHU_IDD-CovidSP', 'Karlen-pypm',
        'LANL-GrowthRate', 'LNQ-ens1', 'MIT_CritData-GBCF',
        'RobertWalraven-ESG', 'SteveMcConnell-CovidComplete',
        'UA-EpiCovDA', 'UMass-MechBayes', 'USC-SI_kJalpha',
        'epiforecasts-ensemble1'], dtype=object),
 '1 wk ahead inc case': array(['BPagano-RtDriven', 'CEID-Walk', 'COVIDhub-baseline',
        'Covid19Sim-Simulator', 'CovidAnalytics-DELPHI',
        'IEM_MED-CovidProject', 'JHUAPL-Bucky', 'JHU_CSSE-DECOM',
        'JHU_IDD-CovidSP', 'Karlen-pypm', 'LANL-GrowthRate', 'LNQ-ens1',
        'RobertWalraven-ESG', 'USC-SI_kJalpha', 'UVA-Ensemble'],
       dtype=object),
 '1 wk ahead inc death': array(['BPagano-RtDriven', 'CEID-Walk', 'COVIDhub-baseline',
        'Covid19Sim-Simulator', 'CovidAnalytics-DELPHI', 'DDS-NBDS',
        'IEM_MED-CovidProject', 

In [61]:
df_test.shape

(246272, 8)

In [62]:
df_test = df_test[df_test.apply(lambda x: x.model in (available_train_models[x.target]), axis=1)]

In [63]:
df_test.head()

,forecast_date,target,target_end_date,quantile,type,value,location,model
0,2021-05-23,1 wk ahead inc death,2021-05-29,0.010,quantile,2063.07986,US,BPagano-RtDriven
1,2021-05-23,1 wk ahead inc death,2021-05-29,0.025,quantile,2268.76972,US,BPagano-RtDriven
2,2021-05-23,1 wk ahead inc death,2021-05-29,0.050,quantile,2452.64561,US,BPagano-RtDriven
3,2021-05-23,1 wk ahead inc death,2021-05-29,0.100,quantile,2667.95825,US,BPagano-RtDriven
4,2021-05-23,1 wk ahead inc death,2021-05-29,0.150,quantile,2814.17454,US,BPagano-RtDriven


In [64]:
df.groupby(['target'])['model'].unique()

target
1 wk ahead cum death    [BPagano-RtDriven, CEID-Walk, COVIDhub-baselin...
1 wk ahead inc case     [BPagano-RtDriven, CEID-Walk, COVIDhub-baselin...
1 wk ahead inc death    [BPagano-RtDriven, CEID-Walk, COVIDhub-baselin...
2 wk ahead cum death    [BPagano-RtDriven, CEID-Walk, COVIDhub-baselin...
2 wk ahead inc case     [BPagano-RtDriven, CEID-Walk, COVIDhub-baselin...
2 wk ahead inc death    [BPagano-RtDriven, CEID-Walk, COVIDhub-baselin...
3 wk ahead cum death    [BPagano-RtDriven, CEID-Walk, COVIDhub-baselin...
3 wk ahead inc case     [BPagano-RtDriven, CEID-Walk, COVIDhub-baselin...
3 wk ahead inc death    [BPagano-RtDriven, CEID-Walk, COVIDhub-baselin...
4 wk ahead cum death    [BPagano-RtDriven, CEID-Walk, COVIDhub-baselin...
4 wk ahead inc case     [BPagano-RtDriven, CEID-Walk, COVIDhub-baselin...
4 wk ahead inc death    [BPagano-RtDriven, CEID-Walk, COVIDhub-baselin...
Name: model, dtype: object

# Export

In [65]:
df.to_csv('data/df_train_{}.csv'.format(str(test_date.date())), index=False)

In [66]:
df_test.to_csv('data/df_test_{}.csv'.format(str(test_date.date())), index=False)